In [8]:
# 딱 한번만 돌릴것
from CrawlingModule import YoutubeBulider
# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')
# API_KEY 할당
API_KEY = KEY_FILE.readline()

# YoutbeModule 사용
bulider = YoutubeBulider(API_KEY)

bulider.search_channelId('김성회의 G식백과')
bulider.get_videoId_in_channel(channelId=bulider.get_channel_id)

AttributeError: 'YoutubeBulider' object has no attribute 'get_channel_id'

In [5]:
import pandas as pd

In [6]:
b = bulider.get_categoryId_in_channel(videoId_list=video_list)
b.get_views_count_list()

[['728784', '9056'], ['596294', '8160'], ['165951', '3946'], ['57757', '523'], ['37770', '478'], ['148364', '2951'], ['135475', '3484'], ['192339', '3244'], ['75736', '2018'], ['148581', '3628'], ['453654', '6945'], ['184864', '3566'], ['447700', '6819'], ['179044', '3738'], ['113986', '1700'], ['143673', '2795'], ['219013', '3358'], ['300113', '5206'], ['737791', '6745'], ['96914', '4624'], ['284768', '4884'], ['217655', '3491'], ['730938', '9326'], ['129512', '2415'], ['93285', '2084'], ['268577', '4482'], ['35727', '872'], ['159708', '2651'], ['52452', '867'], ['76491', '1454'], ['169215', '3527'], ['211401', '4347'], ['127430', '1924'], ['339846', '4859'], ['248759', '2922'], ['171428', '3016'], ['410035', '8245'], ['43526', '1161'], ['269372', '3592'], ['775125', '12743'], ['130224', '2038'], ['125174', '2204'], ['131465', '2783'], ['359964', '4625'], ['90326', '2668'], ['430012', '22840'], ['112216', '2684'], ['407533', '5739'], ['273110', '6647'], ['282678', '12035'], ['445956',

AttributeError: 'list' object has no attribute 'get_views_count_list'

In [181]:
df = pd.DataFrame(
    {
        'video_id' : video_list, 
        'category_id' : list(map(int, categoryId_list)),
    }
)
# df.category_id = df.category_id.astype(int)


In [217]:
infile = open('./category_id_list.txt', 'r').readlines()

key_value_dict = {}

for line in infile:
    key_value_list = line.split(' - ')
    key_value_dict[int(key_value_list[0])] = key_value_list[1][:-1]


df['category_name'] = df.category_id.apply(lambda x : key_value_dict[x])

In [219]:
df.to_csv('./데이터/videoId_and_categoryId_lists.csv', index=False)

In [60]:
df.category_id.value_counts()

,video_id,category_id,category_name
0,Ynwm92g2hxQ,20,20
1,a4abVmaMj5A,20,20
2,TUTMzIP_Mv0,20,20
3,zrN-ybe49lg,20,20
4,PuRqyxLxTTc,20,20
...,...,...,...
95,u9zKB3s-R1A,20,20
96,De0mePZH0rI,20,20
97,Bb43QlfkCAE,20,20
98,wEa_8r0R3P8,20,20
